In [1]:
import numpy as np
from numpy import testing

In [2]:
%run numerics.ipynb

In [3]:
class MPDATA:
    def __init__(self, nr, r_min, r_max, dt, psi_r_lambda, coord, opts):
        self._nm = numerics()
        self.opts = opts
        
        assert r_min.units == r_max.units
        self.r_units = r_min.units
        self.t_units = dt.units
        
        self._n = 0

        #   |-----o-----|-----o--...
        # i-1/2   i   i+1/2   i+1
        # x_min     x_min+dx
        
        self._i = slice(1, nr+1)
        
        # cell-border stuff
        self._im = self._i % self._nm.hlf
        self._xm, self._dx = np.linspace(
            coord.x(r_min).magnitude, 
            coord.x(r_max).magnitude, 
            nr+1, 
            retstep=True
        )
        self._rm = coord.r(self._xm).to(self.r_units).magnitude
        self._Gm = coord.dx_dr(self._rm * self.r_units).magnitude
        self._GC = np.empty_like(self._Gm)
        
        if opts["n_it"] > 1:
            self._GC_tmp = np.empty_like(self._Gm)
        
        # cell-centered stuff
        self._x = np.linspace(
            self._xm[0] - self._dx/2,
            self._xm[-1] + self._dx/2, 
            nr+2 
        )
        self._r = coord.r(self._x).to(self.r_units).magnitude
        
        self._G = np.empty(nr+2)
        self._G = coord.dx_dr(self._r * self.r_units).magnitude

        self._psi = np.empty_like(self._G)
        self._psi = (self._psi, self._psi.copy())
        self._psi[-1][self._i] = psi_r_lambda(self._r[self._i] * self.r_units).magnitude 
       
        # dt, dr
        self._dt = dt.magnitude
        self._dr = np.diff(self._r)
                
    @property
    def psi(self):
        return self._psi[self._n+1][self._i]

    @property
    def r(self):
        return self._r[self._i] * self.r_units
    
    def _bcond(self):
        # quick-n-dirty zero-gradient (extrapolated) boundary condition
        il = (i-nm.hlf).start
        ir = il + 1
        ii = i.start
        psi[n][ii - 1] = psi[n][ii] + Gm[il] * dr[il] / (Gm[ir] * dr[ir]) * (psi[n][ii] - psi[n][ii+1])

        psi[n][i.stop] = -1 # flagging as in principle not used!
        GC[(i-nm.hlf).start] = 0 # TODO?        
        
    def step(self, drdt_r_lambda):        
        # for convenience (TODO!)
        G, Gm, psi, i, im, nm, dr = self._G, self._Gm, self._psi, self._i, self._im, self._nm, self._dr
        
        # integration
        for it in range(self.opts["n_it"]):
            self._n = (self._n+2) % 2 - 1
            n = self._n
            
            if it == 0:
                self._GC[im] = Gm[im] * drdt_r_lambda(self._rm).to(self.r_units / self.t_units).magnitude * self._dt / dr 
                GC = self._GC
            else:
                self._GC_tmp = nm.GC_antidiff(self.opts, psi[n], GC, G, im) 
                GC = self._GC_tmp
                
            testing.assert_array_less(np.amax(GC[im]), 1)
            nm.upwind(psi, GC, G, n, i)
        
            # positive definiteness
            testing.assert_array_less(0, np.amin(psi[n+1][i]))
        
            # conservativeness
            bcflux = 0
            bcflux += psi[n][(i-nm.one).start] * GC[(i-nm.hlf).start] / G[i.start]
            bcflux -= psi[n][i.stop-1] * GC[(i+nm.hlf).stop - 1] / G[(i-nm.one).stop-1]

            testing.assert_almost_equal(
                desired = 1, 
                actual = np.dot(psi[n+1][i], dr[i]) / (np.dot(psi[n][i], dr[i]) + bcflux),
                decimal = 2 # TODO!!!
            ) 
